In [1]:
from pyspark.ml.feature import VectorAssembler

In [2]:
BD = "proyecto_final"

In [3]:
sentencia_select = f"SELECT {BD}.universidades.posicion_hindex, {BD}.grados.empleabilidad, {BD}.grados.nota_corte_ant, {BD}.grados.nota_corte FROM {BD}.grados"
sentencia_join = f"""INNER JOIN {BD}.facultades ON {BD}.facultades.id = {BD}.grados.id_fac
                     INNER JOIN {BD}.universidades ON {BD}.facultades.id_univ = {BD}.universidades.id
                     INNER JOIN {BD}.campos ON {BD}.facultades.id_campo = {BD}.campos.id
                     """
sentencia_sql = f"""{sentencia_select} {sentencia_join}"""

grados_df = spark.sql(sentencia_sql).dropna()
grados_df.show()

2023-07-26 23:25:00,742 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2023-07-26 23:25:00,742 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
2023-07-26 23:25:02,297 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---------------+-------------+--------------+----------+
|posicion_hindex|empleabilidad|nota_corte_ant|nota_corte|
+---------------+-------------+--------------+----------+
|             51|         72.3|           5.0|       5.0|
|             51|         57.5|          5.43|       5.0|
|             51|         77.9|           5.0|       5.0|
|             51|         73.0|           5.0|       5.0|
|             51|         84.4|         10.76|     10.03|
|             51|         85.5|          7.75|       8.2|
|             51|         81.0|           5.0|       5.0|
|             51|         94.2|          8.62|      8.04|
|             51|         89.3|           5.0|       5.0|
|             51|         80.5|          7.77|      5.15|
|             51|         84.2|          9.32|       9.7|
|             51|         79.5|          8.52|      8.97|
|             51|         74.9|          11.4|     11.31|
|             51|         71.6|         11.15|     11.35|
|             

In [4]:
vector_assembler = VectorAssembler(inputCols = ['posicion_hindex', 'empleabilidad', 'nota_corte_ant'],
                                   outputCol = 'features')
vnotas_df = vector_assembler.transform(grados_df)
vnotas_df = vnotas_df.select(['features', 'nota_corte'])
vnotas_df.show(3)

+----------------+----------+
|        features|nota_corte|
+----------------+----------+
| [51.0,72.3,5.0]|       5.0|
|[51.0,57.5,5.43]|       5.0|
| [51.0,77.9,5.0]|       5.0|
+----------------+----------+
only showing top 3 rows



In [5]:
splits = vnotas_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [6]:
from pyspark.ml.regression import LinearRegression, LinearRegressionModel

lr = LinearRegression(featuresCol='features', labelCol='nota_corte', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
lr_model.summary.r2

2023-07-26 23:25:08,318 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2023-07-26 23:25:08,318 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


0.8798861666204095

In [7]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","nota_corte","features").show(5)

+------------------+----------+----------------+
|        prediction|nota_corte|        features|
+------------------+----------+----------------+
| 5.495429816441206|       5.0| [5.0,47.8,5.07]|
|11.272919971323471|     12.16|[5.0,48.0,12.12]|
| 9.928936445932418|     10.44|[5.0,53.3,10.48]|
| 5.511819859433779|      5.07| [5.0,53.6,5.09]|
| 5.610160117389222|      5.18| [5.0,59.5,5.21]|
+------------------+----------+----------------+
only showing top 5 rows



In [8]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="nota_corte", metricName="r2")
print("R Squared (R2) ON TEST DATA = %g" % lr_evaluator.evaluate(lr_predictions))
print("RMSE ON TEST DATA = %g" % lr_model.evaluate(test_df).rootMeanSquaredError)

R Squared (R2) ON TEST DATA = 0.884771
RMSE ON TEST DATA = 0.888632


In [9]:
lr_model.save('./Models/regresion_notas')

In [10]:
new_model = LinearRegressionModel.load('./Models/regresion_notas')
new_predictions = new_model.transform(test_df)

In [11]:
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="nota_corte", metricName="r2")
print("R Squared (R2) ON TEST DATA = %g" % lr_evaluator.evaluate(lr_predictions))
print("RMSE ON TEST DATA = %g" % new_model.evaluate(test_df).rootMeanSquaredError)

R Squared (R2) ON TEST DATA = 0.884771
RMSE ON TEST DATA = 0.888632
